# DNS 管理
本示例

## 预设变量（请替换为你的值）
- `YOUR_DOMAIN_NAME`: 你的主域名，例如 `example.com`
- `YOUR_RR`: 你的记录前缀（主机记录），例如 `www`
- `RECORD_TYPE`: 记录类型，例如 `A` 或 `TXT`
- `RECORD_VALUE`: 记录值，例如 `1.2.3.4` 或某个验证字符串

In [ ]:
PROVIDER = "aliyun"  # 可选："aliyun" 或 "tencent"
YOUR_DOMAIN_NAME = "YOUR_DOMAIN_NAME"
YOUR_RR = "YOUR_RR"
RECORD_TYPE = "A"  # 例如："A"、"CNAME"、"TXT"、"AAAA"...
RECORD_VALUE = "1.2.3.4"  # 例如：IP 地址或 TXT 值

## 初始化客户端
优先从 `CHATTOOL_CONFIG_DIR` 下的 `aliyun.env` 或 `tencent.env` 加载密钥，也支持直接使用环境变量。
需要的环境变量示例：
- 阿里云：`ALIBABA_CLOUD_ACCESS_KEY_ID`、`ALIBABA_CLOUD_ACCESS_KEY_SECRET`、可选 `ALIBABA_CLOUD_REGION_ID`
- 腾讯云：`TENCENT_SECRET_ID`、`TENCENT_SECRET_KEY`

In [ ]:
from dotenv import load_dotenv
from chattool.const import CHATTOOL_CONFIG_DIR
from chattool.tools import AliyunDNSClient, TencentDNSClient

if PROVIDER == "aliyun":
    load_dotenv(CHATTOOL_CONFIG_DIR / "aliyun.env")
    client = AliyunDNSClient()
elif PROVIDER == "tencent":
    load_dotenv(CHATTOOL_CONFIG_DIR / "tencent.env")
    client = TencentDNSClient()
else:
    raise ValueError("PROVIDER 只能是 'aliyun' 或 'tencent'")

## 公共 API 演示
以下方法在两家云上都统一支持，参数与返回结构尽量保持一致：
- `describe_domains()`：获取账户下域名列表
- `describe_domain_records(domain_name)`：获取某域名的解析记录列表
- `describe_subdomain_records(sub_domain)`：获取某子域名的解析记录列表（例如 `www.example.com`）
- `add_domain_record(domain_name, rr, type_, value, ttl=600)`：创建记录，返回记录 ID
- `set_record_value(domain_name, rr, type_, value, ttl=600)`：设置记录值（不存在则创建，存在则更新），返回是否成功
- `delete_record_value(domain_name, rr)`：删除指定子域名的记录，返回是否成功

In [ ]:
# 1) 查询账户域名列表
domains = client.describe_domains()
len(domains), domains[:3]  # 预览前几个

In [ ]:
# 2) 查询某域名的解析记录
records = client.describe_domain_records(YOUR_DOMAIN_NAME)
len(records), records[:5]

In [ ]:
# 3) 查询某子域名的解析记录，例如 www.example.com
sub_domain = f"{YOUR_RR}.{YOUR_DOMAIN_NAME}"
sub_records = client.describe_subdomain_records(sub_domain)
len(sub_records), sub_records[:5]

## 写操作示例（谨慎执行）
以下操作会修改你域名下的解析记录，请确保使用测试域名或明确影响再执行。

In [ ]:
# 4) 创建记录（返回记录ID）
record_id = client.add_domain_record(YOUR_DOMAIN_NAME, YOUR_RR, RECORD_TYPE, RECORD_VALUE, ttl=600)
record_id

In [ ]:
# 5) 设置记录值（不存在则创建，存在则更新）
ok = client.set_record_value(YOUR_DOMAIN_NAME, YOUR_RR, RECORD_TYPE, RECORD_VALUE, ttl=600)
ok

In [ ]:
# 6) 删除指定子域名的记录
ok = client.delete_record_value(YOUR_DOMAIN_NAME, rr=YOUR_RR)
ok

## 说明与提示
- 如果需要动态 IP 更新或证书自动化，请参考项目 `docs/dns.md` 的 CLI 章节。
- 返回的数据结构可能包含厂商特有字段，公共示例中只依赖通用键值。
- 若报权限或配额错误，请检查密钥是否具备对应的 API 权限。